<a href="https://colab.research.google.com/github/gmestrallet/BasicSpatialRNAseq/blob/main/BasicSpatialRNAseq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates how to do basic visium spatial RNAseq analysis using scanpy with data stored in google drive.
More information available https://scanpy.readthedocs.io/en/stable/tutorials/spatial/basic-analysis.html

In [ ]:
#Mount Google Drive to access your files, if they are stored there.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Set the path where you want to store the files (use your own directory).
import os

In [ ]:
#Replace 'RNAseq_folder' with the path to the folder in your Google Drive or use '/content/' for local storage.
rna_seq_path = '/content/drive/My Drive/RNAseq_folder'
os.chdir(rna_seq_path)

In [ ]:
#Install necessary libraries and import
!pip install scanpy  # Make sure scanpy is installed
!pip install anndata  # Make sure anndata is installed
!pip install scanorama  # Make sure scanorama is installed
!pip install h5py  # Make sure h5py is installed
!pip install cupy-cuda11x  # Install cupy for CUDA 11.x (change if using a different CUDA version)
!pip install leidenalg  # Make sure leidenalg is installed
import scanpy as sc
import anndata as an
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scanorama
import h5py
import cupy

In [ ]:
sc.logging.print_versions()
sc.set_figure_params(facecolor="white", figsize=(8, 8))
sc.settings.verbosity = 3

In [ ]:
#Read the filtered feature barcode matrix (gene expression data such as filtered_feature_bc_matrix.h5)
adata = sc.read_visium("/content/drive/My Drive/RNAseq_folder")
adata.var_names_make_unique()
adata.var["mt"] = adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)

In [ ]:
adata

In [ ]:
#QC and preprocessing, we perform basic filtering of spots based on total counts and expressed genes
fig, axs = plt.subplots(1, 4, figsize=(15, 4))
sns.histplot(adata.obs["total_counts"], kde=False, ax=axs[0])
sns.histplot(
    adata.obs["total_counts"][adata.obs["total_counts"] < 10000],
    kde=False,
    bins=40,
    ax=axs[1],
)
sns.histplot(adata.obs["n_genes_by_counts"], kde=False, bins=60, ax=axs[2])
sns.histplot(
    adata.obs["n_genes_by_counts"][adata.obs["n_genes_by_counts"] < 4000],
    kde=False,
    bins=60,
    ax=axs[3],
)

In [ ]:
sc.pp.filter_cells(adata, min_counts=100)
sc.pp.filter_cells(adata, max_counts=35000)
adata = adata[adata.obs["pct_counts_mt"] < 20].copy()
print(f"#cells after MT filter: {adata.n_obs}")
sc.pp.filter_genes(adata, min_cells=10)

In [ ]:
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=2000)

In [ ]:
#Manifold embedding and clustering based on transcriptional similarity
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(
    adata, key_added="clusters", directed=False, n_iterations=2
)

In [ ]:
#Visualization in spatial coordinates
plt.rcParams["figure.figsize"] = (4, 4)
sc.pl.umap(adata, color=["total_counts", "n_genes_by_counts", "clusters"], wspace=0.4)

In [ ]:
adata.obsm['spatial'] = adata.obsm['spatial'].astype(int)
plt.rcParams["figure.figsize"] = (8, 8)
sc.pl.spatial(adata, img_key="hires", color=["total_counts", "n_genes_by_counts"])

In [ ]:
#Visualization of clusters in spatial coordinates
sc.pl.spatial(adata, img_key="hires", color="clusters", size=1.5)

In [ ]:
#Visualization of genes in spatial coordinates. replace with your genes of interest. For human genes write genes' names in capital letters.
sc.pl.spatial(adata, img_key="hires", color=['Cd4','Cd8a'])